In [1]:
import os
import gc
import re
import sys
import csv
import time
import tqdm
import json
import numpy as np
import pandas as pd
from copy import deepcopy
from pathlib import Path
from pprint import pprint
from typing import Optional, Any, List, Union, Tuple, Dict, Callable
from collections import defaultdict, OrderedDict, Counter

import openai
import asyncio
from tqdm.asyncio import tqdm_asyncio

1) Install https://github.com/InternLM/lmdeploy/tree/main
2) run: lmdeploy serve api_server Qwen/Qwen2.5-14B-Instruct  --server-port 23333

In [2]:
from openai import OpenAI, AsyncOpenAI

client = AsyncOpenAI(api_key='YOUR_API_KEY', base_url='http://0.0.0.0:23333/v1')

In [3]:
model_names = await client.models.list()
MODEL_NAME = model_names.data[0].id
MODEL_NAME

'unsloth/llama-3-8b-Instruct'

In [4]:
question = "Who pours liquid from a plastic container into a ziploc bag containing meat pieces?"
answer = "Man"
pred = "Someone"

In [5]:
messages=[
        {
            "role": "system",
            "content": 
                "You are an intelligent chatbot designed for evaluating the correctness of generative outputs for question-answer pairs. "
                "Your task is to compare the predicted answer with the correct answer and determine if they match meaningfully. Here's how you can accomplish the task:"
                "------"
                "##INSTRUCTIONS: "
                "- Focus on the meaningful match between the predicted answer and the correct answer.\n"
                "- Consider synonyms or paraphrases as valid matches.\n"
                "- Evaluate the correctness of the prediction compared to the answer."
        },
        {
            "role": "user",
            "content":
                "Please evaluate the following video-based question-answer pair:\n\n"
                f"Question: {question}\n"
                f"Correct Answer: {answer}\n"
                f"Predicted Answer: {pred}\n\n"
                "Provide your evaluation only as a yes/no and score, where the evaluation is 'yes' if the Predicted Answer and Correct Answer have the same meaning in the context of the Question, 'no' in other cases; the score is an integer value between 0 and 5, with 5 indicating the highest meaningful match. "
                "Please generate the response in the form of a Python dictionary string with keys 'pred' and 'score', where value of 'pred' is  a string of 'yes' or 'no' and value of 'score' is in INTEGER, not STRING."
                "DO NOT PROVIDE ANY OTHER OUTPUT TEXT OR EXPLANATION. Only provide the Python dictionary string. "
                "For example, your response should look like this: {'pred': 'yes', 'score': 4.8}."
        }
    ]
pprint(messages)

[{'content': 'You are an intelligent chatbot designed for evaluating the '
             'correctness of generative outputs for question-answer pairs. '
             'Your task is to compare the predicted answer with the correct '
             "answer and determine if they match meaningfully. Here's how you "
             'can accomplish the task:------##INSTRUCTIONS: - Focus on the '
             'meaningful match between the predicted answer and the correct '
             'answer.\n'
             '- Consider synonyms or paraphrases as valid matches.\n'
             '- Evaluate the correctness of the prediction compared to the '
             'answer.',
  'role': 'system'},
 {'content': 'Please evaluate the following video-based question-answer pair:\n'
             '\n'
             'Question: Who pours liquid from a plastic container into a '
             'ziploc bag containing meat pieces?\n'
             'Correct Answer: Man\n'
             'Predicted Answer: Someone\n'
            

In [6]:
response = await client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
    temperature=0.0,
    max_completion_tokens=250,
    # extra_body={"guided_json": answer_schema, "guided_decoding_backend": "outlines"},
)
print(response)

ChatCompletion(id='1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="{'pred': 'yes', 'score': 5}", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738947859, model='unsloth/llama-3-8b-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=306, total_tokens=318, completion_tokens_details=None, prompt_tokens_details=None))


In [7]:
import json
file = open("sample_set.json")
pred_contents = json.load(file)

In [8]:
import ast

answers = []
scores = []
for ind, elem in enumerate(pred_contents):
    question = elem['q'][0].split('?')[0] + '?'
    answer = elem['real']
    pred = elem['a']
    
    completion = await client.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[
                        {
                            "role": "system",
                            "content": 
                                "You are an intelligent chatbot designed for evaluating the correctness of generative outputs for question-answer pairs. "
                                "Your task is to compare the predicted answer with the correct answer and determine if they match meaningfully. Here's how you can accomplish the task:"
                                "------"
                                "##INSTRUCTIONS: "
                                "- Focus on the meaningful match between the predicted answer and the correct answer.\n"
                                "- Consider synonyms or paraphrases as valid matches.\n"
                                "- Evaluate the correctness of the prediction compared to the answer."
                        },
                        {
                            "role": "user",
                            "content":
                                "Please evaluate the following video-based question-answer pair:\n\n"
                                f"Question: {question}\n"
                                f"Correct Answer: {answer}\n"
                                f"Predicted Answer: {pred}\n\n"
                                "Provide your evaluation only as a yes/no and score, where the evaluation is 'yes' if the Predicted Answer and Correct Answer have the same meaning in the context of the Question, 'no' in other cases; the score is an integer value between 0 and 5, with 5 indicating the highest meaningful match. "
                                "Please generate the response in the form of a Python dictionary string with keys 'pred' and 'score', where value of 'pred' is a string of 'yes' or 'no' and value of 'score' is in INTEGER, not STRING."
                                "DO NOT PROVIDE ANY OTHER OUTPUT TEXT OR EXPLANATION. Only provide the Python dictionary string. "
                                "For example, your response should look like this: {'pred': 'yes', 'score': 4.8}."
                        }
                    ]
    )
    # Convert response to a Python dictionary.
    if (ind%100 ==0):
        print (ind)
        print (print (question, answer, pred))
        print (ast.literal_eval(completion.choices[0].message.content)['pred'])
        print (ast.literal_eval(completion.choices[0].message.content)['score'])
    try:
        answers.append(ast.literal_eval(completion.choices[0].message.content)['pred'])
        scores.append(ast.literal_eval(completion.choices[0].message.content)['score'])
    except:
        answers.append(ast.literal_eval(completion.choices[0].message.content.split("\n\nExplanation")[0])['pred'])
        scores.append(ast.literal_eval(completion.choices[0].message.content.split("\n\nExplanation")[0])['score'])

0
Who pours liquid from a plastic container into a ziploc bag containing meat pieces? Someone Man
None
yes
5
100
Who caught a ball? Man Catcher
None
yes
5
200
Who is hanging on a vine? Woman Cat
None
no
0
300
What does a man operate remote control car through? Office Window
None
no
1
400
What is a young boy playing? Piano Piano
None
yes
5
500
What is a man doing? Sand A man is being painted near a statue which is being worked on and another man is pointing and laughing.
None
no
0
600
Who is carrying another man on his back? Man Person in black robe
None
no
0
700
What is a man doing? Play Playing guitar
None
yes
5
800
Who poured oil on the tomato slices? Man Man
None
yes
5
900
Who pops a wheely in the street? Driver Biker
None
yes
4
1000
Who petting a beaver? Someone Hand
None
yes
4
1100
What is a flight attendant closing the emergency door of? Plane Plane
None
yes
5
1200
What is eating nuts on the floor? Hamster Hamster
None
yes
5
1300
What is playing paino? Someone Hands of a person
N

In [9]:
import numpy as np
np.mean(scores)

3.19953125

In [13]:
import pickle
file = open("gpt_answers.pkl",'rb')
answers_gpt = pickle.load(file)

In [14]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(answers, answers_gpt[:len(answers)])

0.37033607494961596

In [12]:
score_sum = 0
count = 0
yes_count = 0
no_count = 0
for pred in answers:
    if "yes" in pred.lower():
        yes_count += 1
    elif "no" in pred.lower():
        no_count += 1


accuracy = yes_count / (yes_count + no_count)
accuracy

0.63359375